In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


# Digit Recognizer
## Competition Description

MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.

In this competition, The goal is to correctly identify digits from a dataset of tens of thousands of handwritten images.

### Dataset Description:
    
The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ... 026 027

028 029 030 031 ... 054 055

056 057 058 059 ... 082 083

 |   |   |   |  ...  |   |
 
728 729 730 731 ... 754 755

756 757 758 759 ... 782 783


The test data set, (test.csv), is the same as the training set, except that it does not contain the "label" column.

Your submission file should be in the following format: For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

ImageId,Label

1,3

2,7

3,8 

(27997 more lines)

The evaluation metric for this contest is the categorization accuracy, or the proportion of test images that are correctly classified. For example, a categorization accuracy of 0.97 indicates that you have correctly classified all but 3% of the images.

Step 1: Data Loading and Preprocessing

In [43]:
import pandas as pd

# Load the training and test datasets
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')


In [44]:
# Split features (pixel values) and labels (digit values) in the training data
train_features = train_data.drop('label', axis=1)
train_labels = train_data['label']

In [45]:
# Preprocessing: Scale pixel values between 0 and 1
train_features = train_features / 255.0
test_data = test_data / 255.0

In [46]:
# Convert labels to one-hot encoded vectors
num_classes = 10  # Digits from 0 to 9
train_labels = pd.get_dummies(train_labels).values

In [47]:
from sklearn.model_selection import train_test_split
# Split the training data into training and validation sets
train_features, validation_features, train_labels, validation_labels = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

Step 2: Model Selection

For this task, let's use a Convolutional Neural Network (CNN), which is a popular choice for image classification tasks due to their ability to capture spatial patterns in images.

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


Step 3: Model Training

In [49]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_features.values.reshape(-1, 28, 28, 1), train_labels, batch_size=128, epochs=10, validation_split=0.2)


Epoch 1/10
210/210 [==============================] - 10s 42ms/step - loss: 0.4818 - accuracy: 0.8525 - val_loss: 0.1706 - val_accuracy: 0.9521
Epoch 2/10
210/210 [==============================] - 8s 36ms/step - loss: 0.2000 - accuracy: 0.9410 - val_loss: 0.1080 - val_accuracy: 0.9699
Epoch 3/10
210/210 [==============================] - 8s 36ms/step - loss: 0.1353 - accuracy: 0.9589 - val_loss: 0.0891 - val_accuracy: 0.9760
Epoch 4/10
210/210 [==============================] - 7s 35ms/step - loss: 0.1046 - accuracy: 0.9674 - val_loss: 0.0770 - val_accuracy: 0.9789
Epoch 5/10
210/210 [==============================] - 8s 37ms/step - loss: 0.0861 - accuracy: 0.9729 - val_loss: 0.0694 - val_accuracy: 0.9817
Epoch 6/10
210/210 [==============================] - 7s 35ms/step - loss: 0.0754 - accuracy: 0.9775 - val_loss: 0.0651 - val_accuracy: 0.9815
Epoch 7/10
210/210 [==============================] - 7s 35ms/step - loss: 0.0675 - accuracy: 0.9799 - val_loss: 0.0630 - val_accuracy: 0.982

Step 4: Model Evaluation

In [50]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_features.values.reshape(-1, 28, 28, 1), validation_labels)

print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_acc)


263/263 [==============================] - 1s 5ms/step - loss: 0.0530 - accuracy: 0.9848
Validation Loss: 0.0529676117002964
Validation Accuracy: 0.9847618937492371


Step 5: Model Prediction

In [51]:
# Make predictions on the test data
test_predictions = model.predict(test_data.values.reshape(-1, 28, 28, 1))

# Convert predictions to labels
test_labels = test_predictions.argmax(axis=1)


875/875 [==============================] - 4s 5ms/step


Step 6: Submission File Generation

In [52]:
# Generate submission file
submission = pd.DataFrame({'ImageId': range(1, len(test_labels) + 1), 'Label': test_labels})

# Save submission file
submission.to_csv('submission.csv', index=False)
